In [1]:
from math import sqrt
from random import randint
Point = tuple[int, int]

Modular division: $$ \frac{a}{b} \: \% \: m = a(b^{m-2} \: \% \: m) \: \% \: m $$

In [2]:
def mod_div(a: int, b: int, m: int):
	return (a * pow(b, m - 2, m)) % m

Elliptic curve equation: $$ y^{2} = x^{3} + ax + b $$
Weierstrass constraint: $$ 4a^{3} + 27b^{2} \neq 0 $$
Point addition: 
$$ x_{3} = \{\lambda^{2} - x_{1} - x_{2} \} \: mod \: p $$
$$ y_{3} = \{\lambda(x_{1} - x_{3}) - y_{1} \} \: mod \: p $$
$$ \lambda = \frac{y_{2} - y_{1}}{x_{2} - x_{1}} \: mod \: p $$
$$ When \: (x_{1},y_{1}) = (x_{2},y_{2}), $$
$$ \lambda = \frac{3x + a}{2y} \: mod \: p $$

In [3]:
class ECC:
	def __init__(self, a: int, b: int, p: int):
		self.a = a
		self.b = b
		self.p = p
		self.G = self.generate_point()

	def generate_point(self):
		if (4 * (self.a ** 3) + 27 * (self.b ** 2)) == 0:
			raise ValueError("Constraints for Weierstrass equation not met")
		
		x, y = 1, 0
		while True:
			rhs = (x ** 3) + (self.a * x) + self.b
			y = int(sqrt(rhs))
			if y ** 2 == rhs:
				break
			x += 1

		return Point((x, y))

	def add(self, p1: Point, p2: Point):
		x1, y1 = p1
		x2, y2 = p2

		num = y2 - y1
		den = x2 - x1

		# point doubling
		if p1 == p2:
			num = (3 * (x1 ** 2)) + self.a # d(x^3)/dx = 3x^2 + a
			den = 2 * y1 # d(y^2)/dy = 2y
		
		l = mod_div(num, den, self.p)
		x3 = ((l ** 2) - x1 - x2) % self.p
		y3 = ((l * (x1 - x3)) - y1) % self.p

		return Point((x3, y3))

	def mul(self, k: int, p: Point):
		temp = p
		while k != 1:
			temp = self.add(temp, p)
			k -= 1
		return temp

	def sub(self, p1: Point, p2: Point):
		x2, y2 = p2
		return self.add(p1, (x2, -y2))

	def encrypt(self, Pm: Point, Pb: Point):
		x = 5 # randint(1, 10)
		print("Randomly generated value (x):", x)
		# {xG, Pm + xPb}
		return (
			self.mul(x, self.G),
			self.add(Pm, self.mul(x, Pb))
		)
	
	def decrypt(self, Cf: Point, Br: int):
		xf, yf = Cf
		# (Pm + xPb) - Br(xG) = Pm + x(BrG) - BrxG = Pm
		return self.sub(yf, self.mul(Br, xf))

In [4]:
PREDEFINED = True
a, b, p = 1, 2, 11
if not PREDEFINED:
	a = int(input("Enter coefficient a: "))
	b = int(input("Enter coefficient b: "))
	p = int(input("Enter elliptic group mod p: "))

if p < a or p < b:
	raise ValueError("Elliptic group mod p must be greater than a and b")

ecc = ECC(a, b, p)
print("Coefficients: a = {}, b = {}".format(a, b))
print("Elliptic group mod p: {}".format(p))
print("Generated point:", ecc.G)

Coefficients: a = 1, b = 2
Elliptic group mod p: 11
Generated point: (1, 2)


In [5]:
Br = 5 if PREDEFINED else int(input("Enter Private Key (Br): ")) # 5
Pb = ecc.mul(Br, ecc.G)
print("Private Key (Br):", Br)
print("Public Key (Pb):", Pb)

Private Key (Br): 5
Public Key (Pb): (1, 9)


In [6]:
Pm_x, Pm_y = 3, 4

if not PREDEFINED:
	Pm_x = int(input("Enter plaintext x coordinate: ")) # 3
	Pm_y = int(input("Enter plaintext y coordinate: ")) # 4

Pm = Point((Pm_x, Pm_y))
Cf = ecc.encrypt(Pm, Pb)

print("Plaintext", Pm)
print("Ciphertext:", Cf)

Randomly generated value (x): 5
Plaintext (3, 4)
Ciphertext: ((1, 9), (8, 2))


In [7]:
decrypted_text = ecc.decrypt(Cf, Br)
assert decrypted_text == Pm
print("Decrypted text:", decrypted_text)

Decrypted text: (3, 4)
